In [102]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import mcubes
# import pytorch3d

In [4]:
# Define the 3D object
object_3d = torch.tensor([
    [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    [[2, 0, 0], [0, 2, 0], [0, 0, 2]],
    [[3, 0, 0], [0, 3, 0], [0, 0, 3]]
], dtype=torch.float)

# Define the camera parameters for the two views
camera_parameters = [
    {'position': torch.tensor([0, 0, -5], dtype=torch.float),
     'look_at': torch.tensor([0, 0, 0], dtype=torch.float)},
    {'position': torch.tensor([-5, 0, 0], dtype=torch.float),
     'look_at': torch.tensor([0, 0, 0], dtype=torch.float)}
]

# Render the two views
views = []
for parameters in camera_parameters:
    position = parameters['position']
    look_at = parameters['look_at']

    # Compute the view transformation matrix
    up = torch.tensor([0, 1, 0], dtype=torch.float)
    view_matrix = torch.zeros((4, 4))
    view_matrix[:3, 3] = position
    view_matrix[3, 3] = 1

    forward = torch.nn.functional.normalize(look_at - position, dim=0)
    right = torch.cross(up, forward)
    up = torch.cross(forward, right)

    view_matrix[:3, 0] = right
    view_matrix[:3, 1] = up
    view_matrix[:3, 2] = -forward

    # Apply the view transformation to the 3D object
    transformed_object = torch.matmul(view_matrix, object_3d.view(3, -1))
    transformed_object = transformed_object[:3].view(3, 3, 3)

    views.append(transformed_object)

# Visualize the two views
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for i, view in enumerate(views):
    axes[i].imshow(view.numpy())
    axes[i].set_title(f"View {i+1}")

plt.show()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x4 and 3x9)

In [144]:
root = "../../data/shapenet/02773838/1b84dededd445058e44a5473032f38f/input_4.npz"
data = np.load(root)
A = data['tsdf']
# print(type(A))


In [134]:
A_2 = torch.tensor([
    [[1.5, 1, 1], [1, 0, 1], [1, 1, 1]],
    [[2, 0, -2], [-2, -2.5, 2], [2, -2, 0]],
    [[3, 3, 3], [3, 0, 3], [3, 3.5, 3]]
], dtype=torch.float)
A_2


tensor([[[ 1.5000,  1.0000,  1.0000],
         [ 1.0000,  0.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000]],

        [[ 2.0000,  0.0000, -2.0000],
         [-2.0000, -2.5000,  2.0000],
         [ 2.0000, -2.0000,  0.0000]],

        [[ 3.0000,  3.0000,  3.0000],
         [ 3.0000,  0.0000,  3.0000],
         [ 3.0000,  3.5000,  3.0000]]])

In [135]:
A_2 = A_2.numpy()
rotated_tensor_A2 = np.rot90(A_2, axes=(0,1))
print(rotated_tensor_A2)

[[[ 1.   1.   1. ]
  [ 2.  -2.   0. ]
  [ 3.   3.5  3. ]]

 [[ 1.   0.   1. ]
  [-2.  -2.5  2. ]
  [ 3.   0.   3. ]]

 [[ 1.5  1.   1. ]
  [ 2.   0.  -2. ]
  [ 3.   3.   3. ]]]


In [118]:
# B = np.zeros(A.shape)
B = np.random.randint(0, 2, A.shape)
B = np.where(A <= 1e-10, np.multiply(A, B), A)
# B = torch.from_numpy(B)
# A = torch.from_numpy(A)
# B = torch.mul(B, A)
# B = B.numpy()

In [119]:
vertices, triangles = mcubes.marching_cubes(B, 0)
mcubes.export_obj(vertices, triangles, 'outputs/input_corrupted_3.obj')

In [145]:
rotated_tensor = np.rot90(A, axes=(0, 1))
B_rotated_tensor = np.random.randint(0, 2, rotated_tensor.shape)
B_rotated_tensor = np.where(A <= 1e-10, np.multiply(rotated_tensor, B_rotated_tensor), rotated_tensor)

In [146]:
vertices, triangles = mcubes.marching_cubes(B_rotated_tensor, 0)
vertices_1, triangles_1 = mcubes.marching_cubes(rotated_tensor, 0)
mcubes.export_obj(vertices, triangles, 'outputs/input_corrupted_rotated_first.obj')
mcubes.export_obj(vertices_1, triangles_1, 'outputs/input_rotated_first.obj')